In [5]:
import awkward as ak
from pyutils.pyprocess import Processor
from pyutils.pyprint import Print 


In [6]:
# help(Processor)
# nts.mu2e.CeEndpointOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.root

In [7]:
file_name = "../../input/files/nts.mu2e.CeEndpointOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001210_00000580.root"

branches = { 
    "evt" : ["run", "subrun", "event"],
    "crv" : ["crvcoincs.time", "crvcoincs.PEs", "crvcoincs.nHits", "crvcoincs.sectorType",
                "crvcoincs.angle", "crvcoincs.nLayers",  "crvcoincs.timeStart", "crvcoincs.timeEnd",
                "crvcoincs.pos.fCoordinates.fX", "crvcoincs.pos.fCoordinates.fY", "crvcoincs.pos.fCoordinates.fZ"],
    "trk" : ["trk.nactive", "trk.pdg", "trkqual.valid", "trkqual.result"],
    "trkfit" : ["trksegs", "trksegpars_lh"],
    "trkmc" : ["trkmcsim"]
} 

data = Processor().process_data(file_name=file_name, branches=branches)

[pyprocess] ⭐️ Initialised Processor:
	path = 'EventNtuple/ntuple'
	use_remote = False
	verbosity=1
[pyprocess] ✅ Completed process on ../../input/files/nts.mu2e.CeEndpointOnSpillTriggered.MDC2020aw_perfect_v1_3_v06_06_00.001210_00000580.root


In [21]:
def _append_array(data, arr, name):
    """Helper to append arrays/masks to dev field 
    This is useful for debugging and development
    Must be trk or trkfit level
    """
    try:
        if "dev" not in ak.fields(data):
            # Initialise dev field - structure will be determined by first array added
            data = ak.with_field(data, ak.zip({name: arr}, depth_limit=1), "dev")
        else: 
            # Add new field to existing 'dev' record
            new_dev = ak.with_field(data.dev, arr, name)
            data = ak.with_field(data, new_dev, "dev")
        return data
    except Exception as e:
        print("Error appending '{name}' to data array: {e}")
        raise e

In [31]:
def all_downstream(data):
    n = len(data)
    print(f"{n} events before cuts")
    # Segments
    is_downstream_seg = data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"] > 0
    # Tracks 
    all_downstream_segs = ak.all(is_downstream_seg, axis=-1)
    # Events (the actual cut)
    all_downstream_tracks = ak.all(all_downstream_segs, axis=-1)
    # Append
    data = _append_array(data, is_downstream_seg, "is_downstream_seg")
    data = _append_array(data, all_downstream_segs, "all_downstream_segs")
    data = _append_array(data, all_downstream_tracks, "all_downstream_tracks")
    #########
    # Apply inverse cut
    data_cut = data[~all_downstream_tracks]
    #########
    print(f"{len(data_cut)} events failing all downstream tracks cut")
    # Return 
    Print(verbose=True).print_n_events(data["trk"][~all_downstream_tracks], 1)

all_downstream(data)


1482 events before cuts
1481 events failing all downstream tracks cut
[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.nactive: [26, 25, 26, 20]
trk.pdg: [11, -11, 13, -13]
trkqual.valid: [True, True, True, True]
trkqual.result: [0.9, 0.1, 0.1, 0.0]
-------------------------------------------------------------------------------------



In [43]:
def all_downstream_with_trkqual(data):
    n = len(data)
    print(f"{n} events before cuts")
    good_trkqual = data["trk"]["trkqual.result"] > 0.2
    # Segments
    is_downstream_seg = data["trkfit"]["trksegs"]["mom"]["fCoordinates"]["fZ"] > 0
    # Tracks 
    all_downstream_segs = ak.all(is_downstream_seg, axis=-1)
    # Good tracks only
    all_good_downstream_segs = all_downstream_segs & good_trkqual
    # Append
    data = _append_array(data, is_downstream_seg, "is_downstream_seg")
    data = _append_array(data, all_downstream_segs, "all_downstream_segs")
    data = _append_array(data, good_trkqual, "good_trkqual")
    data = _append_array(data, all_good_downstream_segs, "all_good_downstream_segs")
    # Apply inverse cut
    data_cut = ak.copy(data)
    data_cut["trk"] = data["trk"][all_good_downstream_segs]
    has_trk = ak.num(data_cut["trk"]["trk.nactive"]) > 0
    # 
    data_cut = data_cut[has_trk]
    #########
    print(f"{len(data_cut)} events passing all downstream tracks cut")
    # Return 
    Print(verbose=True).print_n_events(data_cut["trk"], 1)
    Print(verbose=True).print_n_events(data_cut["evt"], 1)

all_downstream_with_trkqual(data)


1482 events before cuts
258 events passing all downstream tracks cut
[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
trk.nactive: [26]
trk.pdg: [11]
trkqual.valid: [True]
trkqual.result: [0.9]
-------------------------------------------------------------------------------------

[pyprint] ⭐️ Initialised Print with verbose = True and precision = 1
[pyprint] ⭐️ Printing 1 event(s)...

-------------------------------------------------------------------------------------
run: 1210
subrun: 580
event: 7
-------------------------------------------------------------------------------------

